In [2]:
import numpy as np
import math
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

# Four points on image

HEIGHT_ANGLE=0.87/2 #rad
WIDTH_ANGLE=1.04/2 #rad

old_img = np.zeros((3,5),dtype=float)
old_img[:,0] = [100.0,100.0,1000.0]
old_img[:,1] = [130.0,100.0,800.0] 
old_img[:,2] = [100.0,130.0,1200.0] 
old_img[:,3] = [110.0,110.0,1500.0] 
old_img[:,4] = [70.0,90.0,900.0] 

old_points = np.zeros((3,5),dtype=float)
old_points[0,:] = old_img[2,:]
old_points[1,:] = old_img[2,:]*np.tan(WIDTH_ANGLE*(128-old_img[0,:])/128)
old_points[2,:] = old_img[2,:]*np.tan(HEIGHT_ANGLE*(102.5-old_img[1,:])/102.5)

centroid_old = np.zeros((3),dtype=float)
centroid_old = np.average(old_points,axis=1)
old_points_cent = np.zeros((3,5),dtype=float)
old_points_cent[0,:] = old_points[0,:]-centroid_old[0]
old_points_cent[1,:] = old_points[1,:]-centroid_old[1]
old_points_cent[2,:] = old_points[2,:]-centroid_old[2]

# Rotation Matrix
R_mat = rotation_matrix([1,1,0],0.2)

new_points_cent = np.matmul(R_mat,old_points_cent)
centroid_new = np.matmul(R_mat,centroid_old)+[200,0,0]

new_points = np.zeros((3,5),dtype=float)
new_points[0,:]= new_points_cent[0,:] + centroid_new[0]
new_points[1,:]= new_points_cent[1,:] + centroid_new[1]
new_points[2,:]= new_points_cent[2,:] + centroid_new[2]


new_img = np.zeros((3,5),dtype=float)
new_img[2,:] = new_points[0,:]
new_img[1,:] = 102.5-(np.arctan(new_points[2,:]/new_points[0,:])/HEIGHT_ANGLE*102.5)
new_img[0,:] = 128-(np.arctan(new_points[1,:]/new_points[0,:])/WIDTH_ANGLE*128)

print("R_mat ="+str(R_mat))
print("old_points ="+str(old_points))
print("centroid_old ="+str(centroid_old))
print("centroid_new ="+str(centroid_new))
print("old_img ="+str(old_img))
print("new_img ="+str(new_img))


R_mat =[[ 0.99003329  0.00996671  0.14048043]
 [ 0.00996671  0.99003329 -0.14048043]
 [-0.14048043  0.14048043  0.98006658]]
old_points =[[1000.          800.         1200.         1500.          900.        ]
 [ 114.24315868   -6.50014304  137.09179042  109.88342848  216.07616701]
 [  10.61015422    8.48812337 -140.68811333  -47.76003213   47.78873945]]
centroid_old =[1080.          114.15888031  -24.31222568]
centroid_new =[1266.95834867  127.20053164 -159.50937661]
old_img =[[ 100.  130.  100.  110.   70.]
 [ 100.  100.  130.  110.   90.]
 [1000.  800. 1200. 1500.  900.]]
new_img =[[ 102.99331315  127.91434393   98.05438935  108.91867911   80.22861221]
 [ 124.96098776  127.31468943  151.20425739  136.2342138   113.04208145]
 [1192.66243651  993.15426132 1369.64237417 1679.43575987 1099.89691147]]


In [3]:
Y = old_points_cent
print(Y)

X = new_points_cent
print(X)
S = np.matmul(X,np.transpose(Y))

U, Sig_diag, Vt = np.linalg.svd(S)
print(S)
print("U ="+str(U))



Sig = np.diag(Sig_diag)
print("Sig ="+str(Sig))
print(np.matmul(U,np.matmul(Sig,Vt)))
print("Vt ="+str(Vt))
V = np.transpose(Vt)
print("V ="+str(np.transpose(Vt)))
I = np.identity(3)
I[2,2] = np.linalg.det(np.matmul(V,np.transpose(U)))

Rt = np.matmul(V,np.matmul(I,np.transpose(U)))

R = np.transpose(Rt)
print("R ="+str(R))
print("R_mat ="+str(R_mat))
print("Eigvl(R) = "+str(np.linalg.eig(R)))
print("Eigvl(R_mat) = "+str(np.linalg.eig(R_mat)))

[[-8.00000000e+01 -2.80000000e+02  1.20000000e+02  4.20000000e+02
  -1.80000000e+02]
 [ 8.42783699e-02 -1.20659023e+02  2.29329101e+01 -4.27545183e+00
   1.01917287e+02]
 [ 3.49223799e+01  3.28003491e+01 -1.16375888e+02 -2.34478064e+01
   7.21009651e+01]]
[[ -74.29591215 -273.80408735  102.6840255   412.4774112  -167.0614372 ]
 [  -5.61980948 -126.854936     40.2488846     3.24713697   88.9787239 ]
 [  45.4765313    54.53081493 -127.69214457  -82.58280976  110.2676081 ]]
[[298242.4718891   16595.54006276 -45242.38568109]
 [ 26146.46021736  25283.33024511  -2701.8293443 ]
 [-88762.75759287   2087.09959698  28123.85184198]]
U =[[-0.95307535 -0.04219083  0.29977876]
 [-0.08703626 -0.91024519 -0.40481895]
 [ 0.28995182 -0.41191458  0.86386013]]
Sig =[[316958.64708057      0.              0.        ]
 [     0.          25612.25005371      0.        ]
 [     0.              0.          12506.17260764]]
[[298242.4718891   16595.54006276 -45242.38568109]
 [ 26146.46021736  25283.33024511  -270

In [4]:
averaged = np.zeros((3,3),dtype=float)
averaged[0,:] = [0.9772168077, 0.0307319615, -0.1512946923]
averaged[1,:] = [-0.0386528462, 0.9874825385, -0.0228479231]
averaged[2,:] = [0.1440618462, 0.0287891154, 0.9727689231]

print(averaged)
print(np.linalg.eig(averaged))

[[ 0.97721681  0.03073196 -0.15129469]
 [-0.03865285  0.98748254 -0.02284792]
 [ 0.14406185  0.02878912  0.97276892]]
(array([0.97409162+0.15382277j, 0.97409162-0.15382277j,
       0.98928503+0.j        ]), array([[-0.70270338+0.j        , -0.70270338-0.j        ,
        -0.16852374+0.j        ],
       [-0.11591126-0.17213912j, -0.11591126+0.17213912j,
         0.96326226+0.j        ],
       [-0.03805989+0.67947929j, -0.03805989-0.67947929j,
         0.20910661+0.j        ]]))
